In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import geopandas as gpd
import os
from folium import plugins
import re
import io
import selenium
from selenium import webdriver
from PIL import Image
import time
from folium.plugins import MarkerCluster
plt.rcParams['font.family'] = 'NanumGothic'

In [300]:
chromedriver = 'chromedriver.exe'
driver = webdriver.Chrome(chromedriver)

In [2]:
path = r'C:\Users\EonKim\Desktop\github\khu seminar'
os.chdir(path)
file = r'subway\결과-201802\결과-20180209\20180209_환승인원(1시간 단위).csv'
transfer = pd.read_csv(file, encoding = 'cp949')
drop_col = ['조사일자', '출발역', '도착역']

def drop_columns(dataframe, drop_col):
    for col in drop_col:
        del dataframe[col]
    return dataframe

transfer = drop_columns(transfer, drop_col)
transfer = transfer.groupby('출발역명').sum()
index = transfer.index.values.tolist()

In [242]:

location = pd.read_csv('sub_location.csv',encoding = 'cp949', index_col = 0, header =0)
transfer.merge(location, left_index=True, right_index = True).to_csv('subway_location_merge.csv', index = True)
trans = transfer.merge(location, left_index=True, right_index = True)

In [243]:
sub_c = pd.read_csv('subway\지하철역 수.csv', encoding = 'cp949')
sub_count = sub_c.iloc[:,[0,-1]]
sub_count.columns = ['SGG_NM', '역개수']

In [244]:
seoul = gpd.read_file('subway\LARD_ADM_SECT_SGG_11.shp', encoding = 'euc-kr')

In [ ]:
x_center = np.mean(trans.loc[:,'x'])
y_center = np.mean(trans.loc[:,'y'])
sub_location = np.array(trans.loc[:,['x','y']]).tolist()

m = folium.Map(
    location = [x_center, y_center],
    zoom_start = 11)
folium.Choropleth(
    seoul,
    fill_color = 'white',
    fill_opacity = 0.5
).add_to(m)
for x, y in sub_location:
    folium.CircleMarker(
        location = [x, y],
        radius = 2,
        color = 'Red'
    ).add_to(m)
# m.save('subway_location.html')
m

In [350]:
x_center, y_center

(37.52255396703297, 126.9737072967033)

In [245]:
sub_c.iloc[:,1].fillna('0', inplace =True)

In [246]:
sub_c['해당역(호선)'] = sub_c.iloc[:,1].apply(lambda x: re.sub('(\(){1}(\d){1}(/)?((\d){1})?\)', '', x))

In [247]:
sub_c.iloc[1,1] += '서울역'
# sub_c.iloc[2,1] += '서울역'

In [248]:
subway = []
for sub in sub_c.iloc[:,1]:
    subway.extend(sub.replace(' ','').split('/'))

index = []
for idx in transfer.index:
    if idx in subway:
        index.append(idx)

In [249]:
transfer = transfer.loc[index,:].copy()

In [250]:
index = []
for i in range(len(transfer.index)):
    for j in range(len(sub_c)):
        find = set(sub_c.iloc[j,1].replace(' ','').split('/'))
        if transfer.index[i] in find:
            index.append(sub_c.iloc[j,0])
transfer['행정구역'] = index

In [251]:
transfer['SGG_NM'] = index

In [257]:
population = transfer.groupby('행정구역').sum()

In [274]:
population.columns = list(np.arange(4, 24)) + list(np.arange(4))
population['SGG_NM'] = population.index 

In [285]:
for i in range(24):
    m = folium.Map(
    location = [x_center, y_center],
    zoom_start = 11
    )
    folium.Choropleth(
        geo_data = seoul,
        data = population,
        columns = ('SGG_NM', i),
        key_on = 'properties.SGG_NM',
        fill_color = 'OrRd',
        legend_name = f'{i}시 환승인원'
    ).add_to(m) 
    m.save(f'{i}시 환승인원.html')


In [331]:
pop = transfer.groupby('행정구역').sum().sum(axis = 1)
pop['SGG_NM'] = population.index 
pop1 = pd.DataFrame(pop, columns = ['count'])
pop1['SGG_NM'] = population.index

In [311]:
for i in range(24):
    file_name = f'\{i}시 환승인원.html'
    file = r'C:\Users\EonKim\Desktop\github\khu seminar\subway\image\환승인원' + file_name
    browser = webdriver.Chrome('chromedriver.exe')
    browser.get(file)
    time.sleep(4)
    browser.save_screenshot(f'{i}시 환승인원.png')
    browser.quit()

In [349]:
m = folium.Map(
    location = [x_center, y_center],
    zoom_start = 11
)
folium.Choropleth(
    geo_data = seoul,
    data = pop1,
    columns = ('SGG_NM','count'),
    key_on = 'properties.SGG_NM',
    fill_color = 'OrRd',
    legend_name = '환승 인원'
).add_to(m)
# m.save('지하철역 수.html')
m.save('전체환승인원.html')